In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras import backend as K
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
%matplotlib inline
import matplotlib.image as mpimg
import splitfolders
import cv2
import glob



In [2]:

# FIXING IMAGES EXTENSIONS

for name in glob.glob('/car_plates/train_pics/*'):
    if '.JPG' in name:
        os.rename(name,name.replace('JPG','jpg'))


train_dataset=pd.read_csv(r'/car_plates/train_data.csv')
dataset=pd.read_csv(r'/car_plates/PoolLabels.csv')
dataset.head()

,img.name,label
0,car001.jpg,ر ب ل ١ ٨ ٢ ٧
1,car002.jpg,ى ن ر ٤ ٦ ٢ ٥
2,car003.jpg,ع س ط ١ ٣ ٧ ٥
3,car004.jpg,أ ج س ٤ ١ ٤ ٧
4,car005.jpg,ق ج ل ٢ ٧ ٣ ٩


In [3]:
# FUNCTION TO REFORMAT THE LABELS IN DATASET

def formatLabels(labels):
    count = 0
    labels = labels.apply(lambda x: x.replace(' ',''))
    labels = labels.apply(lambda x: x + "+"*(7-len(x)))
    labels = labels.apply(lambda x: list(x))        
    return labels

# FUNCTION TO REFORMAT THE IMAGES NAMES IN DATASET


def checkName(name):
    if 'JPG' in name:
        return name.replace('JPG','jpg')
    elif '.jpg' not in name:
        return name + '.jpg'
    else:
        return name
    
# RUNNING BOTH FUNCTIONS ON DATASET    
    
dataset['img.name']=dataset['img.name'].apply(checkName)
dataset['label'] = formatLabels(dataset.label)    

In [4]:
from keras.layers import StringLookup

arabic_num_alph={
    "أ":0,
    "ا":0,
    "ب":1,
    "ج":2,
    "د":3,
    "ر":4,
    "ز":5,
    "س":6,
    "ص":7,
    "ط":8,
    "ع":9,
    "ف":10,
    "ق":11,
    "ل":12,
    "م":13,
    "ن":14,
    "ه":15,
    "و":16,
    "ي":17,
    "ى":17,
    "١":18,
    "۱":18,
    "٢":19,
    "۲":19,
    "٣":20,
    "۳":20,
    "٤":21,
    "٥":22,
    "٦":23,
    "٧":24,
    "۷":24,
    "٨":25,
    "۸":25,
    "٩":26,
    "۹":26,
    "+":27,
    "ك":28,
    "ح":29}
    # "١":'1',
    # "٢":'2',
    # "٣":'3',
    # "٤":'4',
    # "٥":'5',
    # "٦":'6',
    # "٧":'7',
    # "٨":'8',
    # "٩":'9',
    # '+':'0',
    # "أ":'11',
    # 'ص':'12',
    # 'ف':'13',
    # 'ل':'14',
    # 'ح':'15',
    # 'س':'16',
    # 'د':'17',
    # 'ي':'18',
    # 'ر':'19',
    # 'ز':'19',
    # 'ن':'20',
    # 'ك':'21',
    # "ب":'22',
    # 'ط':'23',
    # 'ا':'11',
    # 'ج':'24',
    # 'ى':'18',
    # 'م':'25',
    # 'ق':'26',
    # 'ع':'27', 
    # 'ه':'28',
    # "و":'29'
    #         }


# MAKING ARABIC LETTERS AND NUMBERS TO ENGLISH NUMBERS AS SYMBOLS

for index, label in enumerate(dataset['label']):
    for char in label:
        if(char in arabic_num_alph.keys()):    
            # label=  list(map(lambda x: x.replace(char, arabic_num_alph.get(char)), label))
            i = label.index(char)
            label=label[:i]+[arabic_num_alph.get(char)]+label[i+1:]
            dataset['label'][index]=label
        else:
            print(index)
            dataset.drop(index=index,inplace=True)
            
            break
           
            
        
    
            
# ONE HOT ENCODIN THE LABELS INTO MATRIX(7,30)

for index, label in enumerate(dataset['label']):
    matrix = np.zeros(shape=(7,30))
    for char in label:
        i = label.index(char)
        column=int(char)
        matrix[i][column]=1
    dataset['label'][index]=matrix
        
            
          
# label 2000 IS CORRUPTED


2244
2367
2368
2369
2370
2371
2372
2373
2374
2375
2376
2377
2378
2379
2380
2381
2382
2383
2384
2385
2386
2387
2388
2389
2390
2391
2392
2393
2394
2395
2396
2397
2398
2399
2400
2401
2402
2403
2404
2405
2406
2407
2408
2409
2410
2411
2412
2413
2414
2415
2416
2417
2418
2419
2420
2421
2422
2423
2424
2425
2426
2427
2428
2429
2430
2431
2432
2433
2434
2435
2436
2437
2438
2439
2440
2441
2442
2443
2444
2445
2446
2447
2448
2449
2450
2451
2452
2453
2454
2455
2456
2457
2458
2459
2460
2461
2462
2463
2464
2465
2466
2467
2468
2469
2470
2471
2472
2473
2474
2475
2476
2477
2478
2479
2480
2481
2482
2483
2484
2485
2486
2487
2488
2489
2490
2491
2492
2493
2494
2495
2496
2497
2498
2499
2500
2501
2502
2503
2504
2505
2506
2507
2508
2509
2510
2511
2512
2513
2514
2515
2516
2758
2894


In [5]:
train_dataset.tail()

,img.name,label


In [6]:


for index in range(1999):
   train_dataset = train_dataset.append(dataset.iloc[index])
  

C:\Users\Ahmed Medhat\AppData\Local\Temp\ipykernel_4352\724131912.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_dataset = train_dataset.append(dataset.iloc[index])
C:\Users\Ahmed Medhat\AppData\Local\Temp\ipykernel_4352\724131912.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_dataset = train_dataset.append(dataset.iloc[index])
C:\Users\Ahmed Medhat\AppData\Local\Temp\ipykernel_4352\724131912.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_dataset = train_dataset.append(dataset.iloc[index])
C:\Users\Ahmed Medhat\AppData\Local\Temp\ipykernel_4352\724131912.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


In [7]:
train_directory='/car_plates/train_pics/'
img_path=[]
count=0
for image in train_dataset['img.name']:
    try:
        img_path.append(train_directory+train_dataset['img.name'][count])
        count+=1
    except:
        print(index)
        continue  

In [8]:
gray_scaled_dir='/car_plates/grayscaled/'
# count=0
# for img in img_path:
#       img = cv2.imread(img,cv2.IMREAD_GRAYSCALE)
#       img = cv2.resize(img, (128, 128))
#       os.chdir(gray_scaled_dir)
#       cv2.imwrite(train_dataset['img.name'][count], img)
#       count+=1

In [9]:
gray_scaled_path=[]
for index in range(1999):
    try:
        gray_scaled_path.append(gray_scaled_dir+train_dataset['img.name'][index])
    except:
        print(index)
        continue 

In [10]:

for index in range(1999):
    try:
        train_dataset['img.name'][index]=gray_scaled_path[index]
    except:
        print(index)
        continue    
# print(index)    

In [11]:
# for index in range(1999):
#     train_dataset['label'][index]=train_dataset['label'][index].tolist()
    
    

In [12]:
train_dataset.head()

,img.name,label
0,/car_plates/grayscaled/car001.jpg,"[[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,..."
1,/car_plates/grayscaled/car002.jpg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,/car_plates/grayscaled/car003.jpg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,/car_plates/grayscaled/car004.jpg,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,/car_plates/grayscaled/car005.jpg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [13]:
len(train_dataset.index)

1999

In [14]:
x=[]
y=train_dataset['label']
count=0
for img in img_path:
      img = cv2.imread(img,cv2.IMREAD_GRAYSCALE)
      img = cv2.resize(img, (128, 128))
      x.append(img)
      # y.append(train_dataset['label'][count])
      count+=1

In [15]:

# Y=[]

# for label in train_dataset['label']:
#     Y.append(label)

In [16]:
# X = []
# def createTrainingData():
    
      
# createTrainingData()

# print(X[0],Y[0])



In [17]:
# from PIL import Image
# plt.figure(figsize = (12,10))
# plt.subplot(1,2,1)

# plt.title(Y[630])
# plt.imshow(X[630],cmap='gray')
# print(X[630])

In [18]:
# X = np.array(X).reshape(-1, 128, 128, 1)

In [19]:
# config_img=[]
# count=0
# for i in range(img_path.__len__()):
#     try:
#         img=cv2.imread(img_path[count])
#         img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#         img=cv2.resize(img,(128,64))
#         config_img.append(img)
#         count=count+1
#     except:
#         print(img_path[count])
#         count=count+1
#         continue    


In [20]:
# from PIL import Image
# plt.figure(figsize = (12,10))
# plt.subplot(1,2,1)

# plt.title(dataset['label'][2])
# plt.imshow(config_img[2])
# print(config_img[2])

In [21]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
   
    
    )
test_datagen = ImageDataGenerator(rescale = 1./255)

In [22]:
# train_generator = train_datagen.flow_from_dataframe(
#     dataframe=train_dataset,
#                   x_col='img.name',
#                   y_col='label',
#                   target_size =(128, 128), 
#                   color_mode='grayscale',
#                   class_mode='categorical'# target_size = input image size
                
#                   )

# # train_generator= train_datagen.flow_from_directory(
# #   x_train
  
# # )

# # test_generator = test_datagen.flow_from_dataframe(
# #                     x_test,
# #                     target_size =(150, 150),
# #                     batch_size = 20,
# #                     class_mode ='binary')

In [23]:
# from keras.layers import Activation, Dense

# model =tf.keras.models.Sequential([
#  tf.keras.layers.Conv2D(32, (3,3), activation='relu' ,
#  input_shape=(128, 64, 3)),
#  tf.keras.layers.MaxPooling2D(2, 2),
#  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#  tf.keras.layers.MaxPooling2D(2,2),
#  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
#  tf.keras.layers.MaxPooling2D(2,2),
#  tf.keras.layers.Flatten(),
#  tf.keras.layers.Dense(512, activation='relu'),
#  tf.keras.layers.Dense(128, activation='relu'),
# #  tf.keras.layers.Dense(1, activation='softmax'),
# #  tf.keras.layers.Dense(1, activation='softmax'),
# #  tf.keras.layers.Dense(1, activation='softmax'),
# #  tf.keras.layers.Dense(1, activation='softmax'),
# #  tf.keras.layers.Dense(1, activation='softmax'),
# #  tf.keras.layers.Dense(1, activation='softmax'),
#  tf.keras.layers.Dense(29, activation='softmax'), 
  
#  ])




In [45]:
x=np.asarray(x).astype('float64').reshape(-1,128,128,1)

In [46]:
x=x/255

In [48]:
x


array([[[[0.00218378],
         [0.00221453],
         [0.0021684 ],
         ...,
         [0.00189158],
         [0.00195309],
         [0.00186082]],

        [[0.00235294],
         [0.00226067],
         [0.00242983],
         ...,
         [0.00206075],
         [0.0020915 ],
         [0.00202999]],

        [[0.00232218],
         [0.00242983],
         [0.00230681],
         ...,
         [0.0021684 ],
         [0.00219915],
         [0.00224529]],

        ...,

        [[0.0013687 ],
         [0.00132257],
         [0.00130719],
         ...,
         [0.00153787],
         [0.00161476],
         [0.00156863]],

        [[0.00126105],
         [0.00132257],
         [0.0013687 ],
         ...,
         [0.00179931],
         [0.00175317],
         [0.00164552]],

        [[0.00075356],
         [0.00076894],
         [0.00076894],
         ...,
         [0.00179931],
         [0.00184544],
         [0.00173779]]],


       [[[0.00199923],
         [0.00198385],
         [0.00

In [49]:
batch_size = 16
nb_classes =4
nb_epochs = 5
img_rows, img_columns = 128, 128
img_channel = 3
nb_filters = 32
nb_pool = 2
nb_conv = 3

In [50]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu,
                           input_shape=(128, 128, 1)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(7,  activation=tf.nn.softmax)
])

In [51]:
model.compile(
    optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 128, 128, 32)      320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 32, 32, 32)        0         
                                                                 
 flatten_1 (Flatten)         (None, 32768)            

In [52]:
model.fit(x=x,y=y, batch_size = batch_size, epochs = nb_epochs, verbose = 1)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).